In [1]:
import requests
def get_claude_completion(prompt):
    url = 'https://runway.devops.rednote.life/openai/bedrock_runtime/model/invoke'
    headers = {
        'token': 'd32974a62c3d471da311064046166621',
        'Content-Type': 'application/json'
    }
    data = {
        "anthropic_version": "bedrock-2023-05-31", 
        "max_tokens": 4096,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()['content'][0]['text']

In [3]:
DIVIDE_SP_1 = """You are given a complex and information-rich instruction. Your task is to decompose this instruction into:
1. One **core instruction** that captures the essence and overall goal of the original instruction.  
2. Several **supplementary instructions** that elaborate on or add details to the core instruction. These supplementary instructions must be strongly related to and dependent on the core instruction — they should not function as independent tasks.

The goal is **not** to split the complex instruction into multiple independent tasks, but to extract a structured representation where **one core instruction summarizes the main task**, and each supplementary instruction provides additional, context-specific clarifications, constraints, or elaborations that support the core instruction.

Ensure that:  
- The **core instruction** stands as a concise and accurate abstraction of the full intent.  
- Each **supplementary instruction** refines or extends the core instruction without diverging into a separate goal.

Use clear and precise language. Do not omit any key information from the original instruction — ensure all content is preserved through the combination of the core and supplementary instructions.

**Example Input**:
"Write a Python script that reads a CSV file containing user information, filters out users under the age of 18, sorts the remaining users by their registration date in descending order, and then saves the result to a new CSV file with UTF-8 encoding."

**Example Output**:
Core Instruction: Write a Python script to process user data from a CSV file.

Supplementary Instructions:
1. Filter out users who are under the age of 18.
2. Sort the remaining users by registration date in descending order.
3. Save the processed data to a new CSV file using UTF-8 encoding.

**Output Format** (strictly follow this Python dictionary format):

{{
    "Core Instruction": <insert core instruction here>,
    "Supplementary Instructions": [<first supplementary instruction>, <second supplementary instruction>, ...]
}}

Now process this instruction as input:
{prompt}
"""

prompt = DIVIDE_SP_1.format(prompt="Given an integer n, an alternating permutation is a permutation of the first n positive integers such that no two adjacent elements are both odd or both even.\nReturn all such alternating permutations sorted in lexicographical order.\n \nExample 1:\n\nInput: n = 4\nOutput: [[1,2,3,4],[1,4,3,2],[2,1,4,3],[2,3,4,1],[3,2,1,4],[3,4,1,2],[4,1,2,3],[4,3,2,1]]\n\nExample 2:\n\nInput: n = 2\nOutput: [[1,2],[2,1]]\n\nExample 3:\n\nInput: n = 3\nOutput: [[1,2,3],[3,2,1]]\n\n \nConstraints:\n\n1 <= n <= 10\n\n")

In [ ]:
get_claude_completion(prompt)

In [ ]:
import json
import numpy as np
with open('/cpfs/user/zhengziwei/workspace/agent/VeRL-Agent/data/collab_code/LeetCodeDataset-v0.3.0-train-claude.jsonl', 'r') as f:
    supp = [json.loads(line.strip())['collab_divide']['Supplementary Instructions'] for line in f.readlines()]
len(supp)

In [ ]:
num_supp = [len(s) for s in supp]
np.min(num_supp), np.max(num_supp), np.mean(num_supp)

In [ ]:
from verl.workers.agent.envs.collab_code.code import code_exec, extract_code_from_string

PY_IMPORTS = "import heapq\nfrom math import floor, gcd\nimport random\nimport sys\nfrom typing import *\nfrom functools import *\nimport collections\nfrom collections import *\nfrom itertools import *\nfrom heapq import *\nfrom bisect import *\nfrom string import *\nimport math\nimport datetime\ninf = float('inf')\n"

response = 

succ, output = code_exec(solution_code + "\n" + ground_truth["functional"])

In [ ]:
import re

def replace_between_question_and_format(text):
    pattern = r"(### Question:\n)(.*?)(\n\n\n\n### Format:)"
    replaced_text = re.sub(pattern, r"\1{prompt}\3", text, flags=re.DOTALL)
    return replaced_text

replace_between_question_and_format("You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests.\n\n### Question:\nThere is a 1-indexed 8 x 8 chessboard containing 3 pieces.\nYou are given 6 integers a, b, c, d, e, and f where:\n\n(a, b) denotes the position of the white rook.\n(c, d) denotes the position of the white bishop.\n(e, f) denotes the position of the black queen.\n\nGiven that you can only move the white pieces, return the minimum number of moves required to capture the black queen.\nNote that:\n\nRooks can move any number of squares either vertically or horizontally, but cannot jump over other pieces.\nBishops can move any number of squares diagonally, but cannot jump over other pieces.\nA rook or a bishop can capture the queen if it is located in a square that they can move to.\nThe queen does not move.\n\n \nExample 1:\n\n\nInput: a = 1, b = 1, c = 8, d = 8, e = 2, f = 3\nOutput: 2\nExplanation: We can capture the black queen in two moves by moving the white rook to (1, 3) then to (2, 3).\nIt is impossible to capture the black queen in less than two moves since it is not being attacked by any of the pieces at the beginning.\n\nExample 2:\n\n\nInput: a = 5, b = 3, c = 3, d = 4, e = 5, f = 2\nOutput: 1\nExplanation: We can capture the black queen in a single move by doing one of the following: \n- Move the white rook to (5, 2).\n- Move the white bishop to (5, 2).\n\n \nConstraints:\n\n1 <= a, b, c, d, e, f <= 8\nNo two pieces are on the same square.\n\n\n\n### Format: You will use the following starter code to write the solution to the problem and enclose your code within delimiters.\n```python\nclass Solution:\n    def minMovesToCaptureTheQueen(self, a: int, b: int, c: int, d: int, e: int, f: int) -> int:\n```\n\n### Answer: (use the provided format with backticks)\n").format(prompt='!!!!!')

In [ ]:
import re
def extract_code_content(content):
    pattern = r'```python\n(.*?)\n```'
    matches = re.findall(pattern, content, flags=re.DOTALL)
    return matches

len(extract_code_content('ddd')) == 0

In [14]:
import json
import numpy as np

def get_result(input_file):
    accs, turns, success_turns = [], [], []
    with open(input_file, 'r') as f:
        for line in f:
            data = json.loads(line)
            try:
                success = data['test_success']
            except:
                success = False
            accs.append(success)
            if success:
                success_turns.append(data['turns'])
            turns.append(data['turns'])
    return np.mean(accs), np.mean(turns), np.mean(success_turns)

In [ ]:
for model in ['7b', '32b', '72b']:
    for user in ['low', 'mid', 'high']:
        input_file = f'./data/collab_code/qwen-{model}-instruct/LeetCodeDataset-v0.3.0-test-auto_dialogue-{user}.jsonl'.format(model=model, user=user)
        acc, turn, success_turn = get_result(input_file)
        print(f'Qwen-{model}-instruct with {user}-user || Accuracy: {acc:.2f}, Turns: {turn:.2f}, Success_Turns: {success_turn:.2f}'.format(model=model, user=user, acc=acc, turn=turn, success_turn=success_turn))

In [6]:
import pandas as pd

df = pd.read_parquet('/cpfs/user/honglingyi/DATA/LLM/VL_Agent/parquets/vl_agent_V1_test_box.parquet')
df = df.to_dict(orient='records')

In [7]:
df[0]

{'data_source': 'vl_agent',
 'prompt': array([{'content': 'The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "zoom_in", "description": "Zoom in and crop the region in the picture.", "parameters": {"type": "object", "properties": {"region": {"type": "string", "description": "The area to be zoomed, in the format \\{"bbox_2d": [x1, y1, x2, y2]\\}"}}, "required": ["region"]}}}\n</tools>\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call>\nThe output format of reasoning process and final answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "<think> reaso

In [9]:
print(df[0]['prompt'])

[{'content': 'The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "zoom_in", "description": "Zoom in and crop the region in the picture.", "parameters": {"type": "object", "properties": {"region": {"type": "string", "description": "The area to be zoomed, in the format \\{"bbox_2d": [x1, y1, x2, y2]\\}"}}, "required": ["region"]}}}\n</tools>\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call>\nThe output format of reasoning process and final answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "<think> reasoning process here </think><answer> final answ

In [ ]:
json_str = df.to_dict(orient='records')
print(json_str)

In [ ]:
json_str[711]['extra_info']['dataset']

In [ ]:
len(json_str)

In [ ]:
in_out = json.loads(json_str[711]['reward_model']['ground_truth'])
if 'functional' in in_out:
    inputs = in_out['functional']
inputs, outputs = in_out['inputs'], in_out['outputs']

In [ ]:
json_str[711]

In [ ]:
idx = 0
for j in json_str:
    if 'functional' in j['reward_model']['ground_truth']:
        idx += 1
idx

In [ ]:
json.loads(json_str[711]['reward_model']['ground_truth'])

In [ ]:
print(df['prompt'][711][1]['content'])

In [ ]:
print(df['extra_info'][711]['reference'])

In [14]:
import sys
def minimize_stdio(inputs, outputs, max_n_tests=8):
    stdin_list = []
    stdout_list = []
    for stdin, stdout in zip(inputs, outputs):
        if isinstance(stdin, list):
            stdin = "\n".join(stdin)
        if isinstance(stdout, list):
            stdout = "\n".join(stdout)
        if sys.getsizeof(stdin) > 4 * 1024:
            continue
        stdout.replace("\r\n", "\n")
        stdin_list.append(stdin)
        stdout_list.append(stdout)

    zipped = sorted(zip(stdin_list, stdout_list), key=lambda x: sys.getsizeof(x[0]))

    if not zipped:
        print("No tests found!")
        return [], []

    sorted_stdin, sorted_stdout = zip(*zipped)
    
    return [{'input': {'stdin': inp}, 'output': {'stdout': oup}} for inp, oup in zip(list(sorted_stdin[:max_n_tests]), list(sorted_stdout[:max_n_tests]))]

In [18]:
import json
# in_out = json.loads(df['reward_model'][711]['ground_truth'])
# inputs, outputs = in_out['inputs'], in_out['outputs']

in_out = json.loads(json_str[711]['reward_model']['ground_truth'])
inputs, outputs = in_out['inputs'], in_out['outputs']

unitests = minimize_stdio(inputs, outputs)

In [ ]:
unitests

In [23]:
unitests.append({'input': {'stdin': '1'}, 'output': {'stdout': '1'}})

In [24]:
from verl.workers.agent.envs.collab_code.sandbox_verify import OJConfig, oj_in_sandbox, OJRequest

code_str = df['extra_info'][711]['reference']
# tests = ground_truth
oj_data = {
    "id": 1,                          # Unique identifier
    "content": '',                     # Problem statement
    "test": unitests
}

result = oj_in_sandbox(OJRequest(
    completion=f"```python\n{code_str}\n```",
    config=OJConfig(language='python', provided_data=oj_data, extra={'run_all_cases': True}, run_timeout=5)),
                    connection_timeout=5)

In [ ]:
result.accepted

In [ ]:
s = """
{
    "Core Instruction": Search for information about the two standard ways to represent a graph in computer science.",
    "Supplementary Instructions": [
        "The representation methods should specifically include adjacency list representation.",
        "The representation methods should also include adjacency matrix representation.",
        "The information should define a graph G = (V, E) where V is a set of vertices and E is a set of edges."
    ]
}
"""

json.loads(s)

In [ ]:
import re

def try_extract_solution(solution_str):
    answer_pattern = r'<answer>(.*?)</answer>'
    matches = list(re.finditer(answer_pattern, solution_str, re.DOTALL))

    if matches:
        final_answer = matches[-1].group(1).strip()
        return final_answer

    return solution_str

CODE_PATTERN = re.compile(r'```(?:\w+)?\n(.*?)\n```', re.DOTALL)

def extract_code_from_string(solution_str):
    solution_str = try_extract_solution(solution_str)
    code_blocks = CODE_PATTERN.findall(solution_str)
    return '\n'.join(code_blocks).strip()

In [ ]:
solution_str = """
Some explanation.

<answer>
This is just a plain answer without any code block.
</answer>
"""

extract_code_from_string(solution_str)